**In this file, I test how parameters contribute/influence to the model.**

In [10]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import metrics

In [11]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [12]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [13]:
batch_size = 32
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20

In [14]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [15]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [16]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [17]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)
    
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

Using real-time data augmentation.


**The following configuration is the reference standard to evaluate model**

**1. No of epochs: 5**

**2. Batch size: 32**

**3. Network configuration**

    a. Number of neurons in a layer: 512
    
    b. Number of layers: 3
    
**4. Learning rate: 0.0001**

**5. Activation functions: relu**

**6. Dropout rates: 0.25**

In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [19]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
optimizer=opt,
metrics=['accuracy',metrics.categorical_accuracy])

In [20]:
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 220s 141ms/step - loss: 1.8086 - acc: 0.3371 - categorical_accuracy: 0.3371 - val_loss: 1.5465 - val_acc: 0.4343 - val_categorical_accuracy: 0.4343
Epoch 2/5
1563/1563 [==============================] - 226s 144ms/step - loss: 1.5566 - acc: 0.4305 - categorical_accuracy: 0.4305 - val_loss: 1.3895 - val_acc: 0.4939 - val_categorical_accuracy: 0.4939
Epoch 3/5
1563/1563 [==============================] - 164s 105ms/step - loss: 1.4387 - acc: 0.4795 - categorical_accuracy: 0.4795 - val_loss: 1.2483 - val_acc: 0.5544 - val_categorical_accuracy: 0.5544
Epoch 4/5
1563/1563 [==============================] - 141s 90ms/step - loss: 1.3477 - acc: 0.5194 - categorical_accuracy: 0.5194 - val_loss: 1.2112 - val_acc: 0.5733 - val_categorical_accuracy: 0.5733
Epoch 5/5
1563/1563 [==============================] - 142s 91ms/step - loss: 1.2748 - acc: 0.5460 - categorical_accuracy: 0.5460 - val_loss: 1.0913 - val_acc: 0.6180 - val_categorical_accu

In [21]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 6s 630us/step
Test loss: 1.09130263615
Test accuracy: 0.618


**Change No. of epochs from 5 to 7**

In [22]:
model_epochs = Sequential()
model_epochs.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_epochs.add(Activation('relu'))
model_epochs.add(Conv2D(32, (3, 3)))
model_epochs.add(Activation('relu'))
model_epochs.add(MaxPooling2D(pool_size=(2, 2)))
model_epochs.add(Dropout(0.25))

model_epochs.add(Conv2D(64, (3, 3), padding='same'))
model_epochs.add(Activation('relu'))
model_epochs.add(Conv2D(64, (3, 3)))
model_epochs.add(Activation('relu'))
model_epochs.add(MaxPooling2D(pool_size=(2, 2)))
model_epochs.add(Dropout(0.25))

model_epochs.add(Flatten())
model_epochs.add(Dense(512))
model_epochs.add(Activation('relu'))
model_epochs.add(Dropout(0.25))
model_epochs.add(Dense(num_classes))
model_epochs.add(Activation('softmax'))

model_epochs.compile(loss='categorical_crossentropy',
optimizer=opt,
metrics=['accuracy',metrics.categorical_accuracy])

model_epochs.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=7,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/7
1563/1563 [==============================] - 138s 89ms/step - loss: 1.8407 - acc: 0.3234 - categorical_accuracy: 0.3234 - val_loss: 1.5726 - val_acc: 0.4348 - val_categorical_accuracy: 0.4348
Epoch 2/7
1563/1563 [==============================] - 134s 86ms/step - loss: 1.5479 - acc: 0.4333 - categorical_accuracy: 0.4333 - val_loss: 1.4684 - val_acc: 0.4771 - val_categorical_accuracy: 0.4771
Epoch 3/7
1563/1563 [==============================] - 134s 86ms/step - loss: 1.4260 - acc: 0.4880 - categorical_accuracy: 0.4880 - val_loss: 1.3567 - val_acc: 0.5188 - val_categorical_accuracy: 0.5188
Epoch 4/7
1563/1563 [==============================] - 135s 87ms/step - loss: 1.3362 - acc: 0.5234 - categorical_accuracy: 0.5234 - val_loss: 1.2653 - val_acc: 0.5560 - val_categorical_accuracy: 0.5560
Epoch 5/7
1563/1563 [==============================] - 136s 87ms/step - loss: 1.2629 - acc: 0.5524 - categorical_accuracy: 0.5524 - val_loss: 1.1400 - val_acc: 0.5980 - val_categorical_accurac

In [23]:
scores = model_epochs.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 6s 577us/step
Test loss: 0.996672880554
Test accuracy: 0.6481


**The accuracy increased and the loss decreased.**

**Change batch size from 32 to 64**

In [24]:
model_batch = Sequential()
model_batch.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_batch.add(Activation('relu'))
model_batch.add(Conv2D(32, (3, 3)))
model_batch.add(Activation('relu'))
model_batch.add(MaxPooling2D(pool_size=(2, 2)))
model_batch.add(Dropout(0.25))

model_batch.add(Conv2D(64, (3, 3), padding='same'))
model_batch.add(Activation('relu'))
model_batch.add(Conv2D(64, (3, 3)))
model_batch.add(Activation('relu'))
model_batch.add(MaxPooling2D(pool_size=(2, 2)))
model_batch.add(Dropout(0.25))

model_batch.add(Flatten())
model_batch.add(Dense(512))
model_batch.add(Activation('relu'))
model_batch.add(Dropout(0.25))
model_batch.add(Dense(num_classes))
model_batch.add(Activation('softmax'))

model_batch.compile(loss='categorical_crossentropy',
optimizer=opt,
metrics=['accuracy',metrics.categorical_accuracy])

model_batch.fit_generator(datagen.flow(x_train, y_train,
    batch_size=64),
    epochs=epochs,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
782/782 [==============================] - 143s 183ms/step - loss: 1.9168 - acc: 0.3022 - categorical_accuracy: 0.3022 - val_loss: 1.6519 - val_acc: 0.4064 - val_categorical_accuracy: 0.4064
Epoch 2/5
782/782 [==============================] - 134s 171ms/step - loss: 1.6479 - acc: 0.4038 - categorical_accuracy: 0.4038 - val_loss: 1.5043 - val_acc: 0.4463 - val_categorical_accuracy: 0.4463
Epoch 3/5
782/782 [==============================] - 137s 175ms/step - loss: 1.5228 - acc: 0.4504 - categorical_accuracy: 0.4504 - val_loss: 1.3821 - val_acc: 0.4949 - val_categorical_accuracy: 0.4949
Epoch 4/5
782/782 [==============================] - 144s 184ms/step - loss: 1.4362 - acc: 0.4823 - categorical_accuracy: 0.4823 - val_loss: 1.2844 - val_acc: 0.5408 - val_categorical_accuracy: 0.5408
Epoch 5/5
782/782 [==============================] - 139s 177ms/step - loss: 1.3771 - acc: 0.5081 - categorical_accuracy: 0.5081 - val_loss: 1.2294 - val_acc: 0.5650 - val_categorical_accuracy: 0.

In [25]:
scores = model_batch.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 6s 600us/step
Test loss: 1.22942170353
Test accuracy: 0.565


**The accuracy decreased and the loss increased.**

**Change number of neurons in a layer from 512 to 1024**

In [27]:
model_neuron = Sequential()
model_neuron.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_neuron.add(Activation('relu'))
model_neuron.add(Conv2D(32, (3, 3)))
model_neuron.add(Activation('relu'))
model_neuron.add(MaxPooling2D(pool_size=(2, 2)))
model_neuron.add(Dropout(0.25))

model_neuron.add(Conv2D(64, (3, 3), padding='same'))
model_neuron.add(Activation('relu'))
model_neuron.add(Conv2D(64, (3, 3)))
model_neuron.add(Activation('relu'))
model_neuron.add(MaxPooling2D(pool_size=(2, 2)))
model_neuron.add(Dropout(0.25))

model_neuron.add(Flatten())
model_neuron.add(Dense(1024))
model_neuron.add(Activation('relu'))
model_neuron.add(Dropout(0.25))
model_neuron.add(Dense(num_classes))
model_neuron.add(Activation('softmax'))

model_neuron.compile(loss='categorical_crossentropy',
optimizer=opt,
metrics=['accuracy',metrics.categorical_accuracy])

model_neuron.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 201s 129ms/step - loss: 1.8242 - acc: 0.3316 - categorical_accuracy: 0.3316 - val_loss: 1.5656 - val_acc: 0.4310 - val_categorical_accuracy: 0.4310
Epoch 2/5
1563/1563 [==============================] - 205s 131ms/step - loss: 1.5396 - acc: 0.4415 - categorical_accuracy: 0.4415 - val_loss: 1.3589 - val_acc: 0.5087 - val_categorical_accuracy: 0.5087
Epoch 3/5
1563/1563 [==============================] - 199s 127ms/step - loss: 1.4042 - acc: 0.4935 - categorical_accuracy: 0.4935 - val_loss: 1.2510 - val_acc: 0.5521 - val_categorical_accuracy: 0.5521
Epoch 4/5
1563/1563 [==============================] - 206s 132ms/step - loss: 1.3142 - acc: 0.5295 - categorical_accuracy: 0.5295 - val_loss: 1.2624 - val_acc: 0.5522 - val_categorical_accuracy: 0.5522
Epoch 5/5
1563/1563 [==============================] - 204s 130ms/step - loss: 1.2398 - acc: 0.5565 - categorical_accuracy: 0.5565 - val_loss: 1.1122 - val_acc: 0.6028 - val_categorical_ac

In [28]:
scores = model_neuron.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 672us/step
Test loss: 1.11216862774
Test accuracy: 0.6028


**The accuracy decreased and the loss increased.**

**Change number of layers from 3 to 4**

In [40]:
model_layer = Sequential()
model_layer.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_layer.add(Activation('relu'))
model_layer.add(Conv2D(32, (3, 3)))
model_layer.add(Activation('relu'))
model_layer.add(MaxPooling2D(pool_size=(2, 2)))
model_layer.add(Dropout(0.25))

model_layer.add(Conv2D(64, (3, 3), padding='same'))
model_layer.add(Activation('relu'))
model_layer.add(Conv2D(64, (3, 3)))
model_layer.add(Activation('relu'))
model_layer.add(MaxPooling2D(pool_size=(2, 2)))
model_layer.add(Dropout(0.25))

model_layer.add(Conv2D(128, (3, 3), padding='same'))
model_layer.add(Activation('relu'))
model_layer.add(Conv2D(128, (3, 3)))
model_layer.add(Activation('relu'))
model_layer.add(MaxPooling2D(pool_size=(2, 2)))
model_layer.add(Dropout(0.25))

model_layer.add(Flatten())
model_layer.add(Dense(512))
model_layer.add(Activation('relu'))
model_layer.add(Dropout(0.25))
model_layer.add(Dense(num_classes))
model_layer.add(Activation('softmax'))

model_layer.compile(loss='categorical_crossentropy',
optimizer=opt,
metrics=['accuracy',metrics.categorical_accuracy])

model_layer.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 185s 118ms/step - loss: 1.9534 - acc: 0.2662 - categorical_accuracy: 0.2662 - val_loss: 1.7499 - val_acc: 0.3603 - val_categorical_accuracy: 0.3603
Epoch 2/5
1563/1563 [==============================] - 188s 120ms/step - loss: 1.6768 - acc: 0.3807 - categorical_accuracy: 0.3807 - val_loss: 1.5011 - val_acc: 0.4550 - val_categorical_accuracy: 0.4550
Epoch 3/5
1563/1563 [==============================] - 180s 115ms/step - loss: 1.5331 - acc: 0.4354 - categorical_accuracy: 0.4354 - val_loss: 1.4156 - val_acc: 0.4855 - val_categorical_accuracy: 0.4855
Epoch 4/5
1563/1563 [==============================] - 184s 118ms/step - loss: 1.4384 - acc: 0.4754 - categorical_accuracy: 0.4754 - val_loss: 1.5135 - val_acc: 0.4640 - val_categorical_accuracy: 0.4640
Epoch 5/5
1563/1563 [==============================] - 188s 120ms/step - loss: 1.3599 - acc: 0.5071 - categorical_accuracy: 0.5071 - val_loss: 1.3383 - val_acc: 0.5321 - val_categorical_ac

In [41]:
score = model_layer.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 8s 781us/step
Test loss: 1.33829700947
Test accuracy: 0.5321


**The accuracy decreased and the loss increased.**

**Change learning rate from 0.0001 to 0.0002**

In [32]:
model_lr = Sequential()
model_lr.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_lr.add(Activation('relu'))
model_lr.add(Conv2D(32, (3, 3)))
model_lr.add(Activation('relu'))
model_lr.add(MaxPooling2D(pool_size=(2, 2)))
model_lr.add(Dropout(0.25))

model_lr.add(Conv2D(64, (3, 3), padding='same'))
model_lr.add(Activation('relu'))
model_lr.add(Conv2D(64, (3, 3)))
model_lr.add(Activation('relu'))
model_lr.add(MaxPooling2D(pool_size=(2, 2)))
model_lr.add(Dropout(0.25))

model_lr.add(Flatten())
model_lr.add(Dense(512))
model_lr.add(Activation('relu'))
model_lr.add(Dropout(0.25))
model_lr.add(Dense(num_classes))
model_lr.add(Activation('softmax'))

model_lr.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',metrics.categorical_accuracy])

model_lr.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 142s 91ms/step - loss: 1.7016 - acc: 0.3767 - categorical_accuracy: 0.3767 - val_loss: 1.4014 - val_acc: 0.4909 - val_categorical_accuracy: 0.4909
Epoch 2/5
1563/1563 [==============================] - 147s 94ms/step - loss: 1.4003 - acc: 0.4952 - categorical_accuracy: 0.4952 - val_loss: 1.1793 - val_acc: 0.5759 - val_categorical_accuracy: 0.5759
Epoch 3/5
1563/1563 [==============================] - 158s 101ms/step - loss: 1.2586 - acc: 0.5521 - categorical_accuracy: 0.5521 - val_loss: 1.0647 - val_acc: 0.6230 - val_categorical_accuracy: 0.6230
Epoch 4/5
1563/1563 [==============================] - 151s 97ms/step - loss: 1.1687 - acc: 0.5841 - categorical_accuracy: 0.5841 - val_loss: 1.0192 - val_acc: 0.6381 - val_categorical_accuracy: 0.6381
Epoch 5/5
1563/1563 [==============================] - 155s 99ms/step - loss: 1.0940 - acc: 0.6129 - categorical_accuracy: 0.6129 - val_loss: 1.0001 - val_acc: 0.6517 - val_categorical_accura

In [33]:
score = model_lr.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 6s 612us/step
Test loss: 1.00013997288
Test accuracy: 0.6517


**The accuracy increased and the loss decreased. The appropriate learning rate can increase the accuracy of model.**

**Change activation function from relu to elu**

In [34]:
model_activation = Sequential()
model_activation.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_activation.add(Activation('elu'))
model_activation.add(Conv2D(32, (3, 3)))
model_activation.add(Activation('elu'))
model_activation.add(MaxPooling2D(pool_size=(2, 2)))
model_activation.add(Dropout(0.25))

model_activation.add(Conv2D(64, (3, 3), padding='same'))
model_activation.add(Activation('elu'))
model_activation.add(Conv2D(64, (3, 3)))
model_activation.add(Activation('elu'))
model_activation.add(MaxPooling2D(pool_size=(2, 2)))
model_activation.add(Dropout(0.25))

model_activation.add(Flatten())
model_activation.add(Dense(512))
model_activation.add(Activation('elu'))
model_activation.add(Dropout(0.25))
model_activation.add(Dense(num_classes))
model_activation.add(Activation('softmax'))

model_activation.compile(loss='categorical_crossentropy',
optimizer=opt,
metrics=['accuracy',metrics.categorical_accuracy])

model_activation.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 146s 93ms/step - loss: 1.7654 - acc: 0.3685 - categorical_accuracy: 0.3685 - val_loss: 1.5085 - val_acc: 0.4675 - val_categorical_accuracy: 0.4675
Epoch 2/5
1563/1563 [==============================] - 149s 95ms/step - loss: 1.5272 - acc: 0.4592 - categorical_accuracy: 0.4592 - val_loss: 1.3895 - val_acc: 0.4973 - val_categorical_accuracy: 0.4973
Epoch 3/5
1563/1563 [==============================] - 153s 98ms/step - loss: 1.4114 - acc: 0.5021 - categorical_accuracy: 0.5021 - val_loss: 1.3085 - val_acc: 0.5349 - val_categorical_accuracy: 0.5349
Epoch 4/5
1563/1563 [==============================] - 156s 100ms/step - loss: 1.3279 - acc: 0.5333 - categorical_accuracy: 0.5333 - val_loss: 1.2035 - val_acc: 0.5741 - val_categorical_accuracy: 0.5741
Epoch 5/5
1563/1563 [==============================] - 155s 99ms/step - loss: 1.2732 - acc: 0.5535 - categorical_accuracy: 0.5535 - val_loss: 1.1427 - val_acc: 0.5988 - val_categorical_accura

In [35]:
score = model_activation.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 7s 669us/step
Test loss: 1.14273164692
Test accuracy: 0.5988


**The accuracy decreased and the loss increased. The activation function 'relu' is better than 'elu' in this model.**

**Change dropout rates from 0.25 to 0.1**

In [36]:
model_dropout = Sequential()
model_dropout.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_dropout.add(Activation('relu'))
model_dropout.add(Conv2D(32, (3, 3)))
model_dropout.add(Activation('relu'))
model_dropout.add(MaxPooling2D(pool_size=(2, 2)))
model_dropout.add(Dropout(0.1))

model_dropout.add(Conv2D(64, (3, 3), padding='same'))
model_dropout.add(Activation('relu'))
model_dropout.add(Conv2D(64, (3, 3)))
model_dropout.add(Activation('relu'))
model_dropout.add(MaxPooling2D(pool_size=(2, 2)))
model_dropout.add(Dropout(0.1))

model_dropout.add(Flatten())
model_dropout.add(Dense(512))
model_dropout.add(Activation('relu'))
model_dropout.add(Dropout(0.1))
model_dropout.add(Dense(num_classes))
model_dropout.add(Activation('softmax'))

model_dropout.compile(loss='categorical_crossentropy',
optimizer=opt,
metrics=['accuracy',metrics.categorical_accuracy])

model_dropout.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/5
1563/1563 [==============================] - 148s 94ms/step - loss: 1.7881 - acc: 0.3474 - categorical_accuracy: 0.3474 - val_loss: 1.5325 - val_acc: 0.4460 - val_categorical_accuracy: 0.4460
Epoch 2/5
1563/1563 [==============================] - 148s 95ms/step - loss: 1.5034 - acc: 0.4556 - categorical_accuracy: 0.4556 - val_loss: 1.3153 - val_acc: 0.5329 - val_categorical_accuracy: 0.5329
Epoch 3/5
1563/1563 [==============================] - 147s 94ms/step - loss: 1.3754 - acc: 0.5058 - categorical_accuracy: 0.5058 - val_loss: 1.1950 - val_acc: 0.5733 - val_categorical_accuracy: 0.5733
Epoch 4/5
1563/1563 [==============================] - 143s 92ms/step - loss: 1.2746 - acc: 0.5483 - categorical_accuracy: 0.5483 - val_loss: 1.1405 - val_acc: 0.5923 - val_categorical_accuracy: 0.5923
Epoch 5/5
1563/1563 [==============================] - 142s 91ms/step - loss: 1.1953 - acc: 0.5757 - categorical_accuracy: 0.5757 - val_loss: 1.0283 - val_acc: 0.6377 - val_categorical_accurac

In [37]:
score = model_dropout.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 6s 590us/step
Test loss: 1.02828486137
Test accuracy: 0.6377


**The accuracy increased and the loss decreased. The appropriate dropout rate can increase the accuracy of model.**

**My best model is in file '7374 Assignment1 Best Model'**

In [44]:
model_best = Sequential()
model_best.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_best.add(Activation('relu'))
model_best.add(Conv2D(32, (3, 3)))
model_best.add(Activation('relu'))
model_best.add(MaxPooling2D(pool_size=(2, 2)))
model_best.add(Dropout(0.1))

model_best.add(Conv2D(64, (3, 3), padding='same'))
model_best.add(Activation('relu'))
model_best.add(Conv2D(64, (3, 3)))
model_best.add(Activation('relu'))
model_best.add(MaxPooling2D(pool_size=(2, 2)))
model_best.add(Dropout(0.1))

model_best.add(Flatten())
model_best.add(Dense(512))
model_best.add(Activation('relu'))
model_best.add(Dropout(0.1))
model_best.add(Dense(num_classes))
model_best.add(Activation('softmax'))

model_best.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',metrics.categorical_accuracy])

model_best.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=100,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/100
1563/1563 [==============================] - 153s 98ms/step - loss: 1.6513 - acc: 0.3980 - categorical_accuracy: 0.3980 - val_loss: 1.3325 - val_acc: 0.5210 - val_categorical_accuracy: 0.5210
Epoch 2/100
1563/1563 [==============================] - 148s 95ms/step - loss: 1.3411 - acc: 0.5206 - categorical_accuracy: 0.5206 - val_loss: 1.1658 - val_acc: 0.5924 - val_categorical_accuracy: 0.5924
Epoch 3/100
1563/1563 [==============================] - 152s 97ms/step - loss: 1.1825 - acc: 0.5838 - categorical_accuracy: 0.5838 - val_loss: 0.9931 - val_acc: 0.6490 - val_categorical_accuracy: 0.6490
Epoch 4/100
1563/1563 [==============================] - 152s 97ms/step - loss: 1.0828 - acc: 0.6185 - categorical_accuracy: 0.6185 - val_loss: 1.0200 - val_acc: 0.6452 - val_categorical_accuracy: 0.6452
Epoch 5/100
1563/1563 [==============================] - 154s 99ms/step - loss: 1.0118 - acc: 0.6440 - categorical_accuracy: 0.6440 - val_loss: 0.9375 - val_acc: 0.6786 - val_categoric

1563/1563 [==============================] - 150s 96ms/step - loss: 0.6464 - acc: 0.7857 - categorical_accuracy: 0.7857 - val_loss: 0.5784 - val_acc: 0.8072 - val_categorical_accuracy: 0.8072
Epoch 42/100
1563/1563 [==============================] - 149s 95ms/step - loss: 0.6449 - acc: 0.7837 - categorical_accuracy: 0.7837 - val_loss: 0.6507 - val_acc: 0.7901 - val_categorical_accuracy: 0.7901
Epoch 43/100
1563/1563 [==============================] - 149s 96ms/step - loss: 0.6426 - acc: 0.7852 - categorical_accuracy: 0.7852 - val_loss: 0.6256 - val_acc: 0.8040 - val_categorical_accuracy: 0.8040
Epoch 44/100
1563/1563 [==============================] - 151s 97ms/step - loss: 0.6468 - acc: 0.7842 - categorical_accuracy: 0.7842 - val_loss: 0.5898 - val_acc: 0.8080 - val_categorical_accuracy: 0.8080
Epoch 45/100
1563/1563 [==============================] - 151s 97ms/step - loss: 0.6475 - acc: 0.7832 - categorical_accuracy: 0.7832 - val_loss: 0.6625 - val_acc: 0.7851 - val_categorical_accur

1563/1563 [==============================] - 151s 96ms/step - loss: 0.7349 - acc: 0.7606 - categorical_accuracy: 0.7606 - val_loss: 0.7338 - val_acc: 0.7739 - val_categorical_accuracy: 0.7739
Epoch 82/100
1563/1563 [==============================] - 150s 96ms/step - loss: 0.7396 - acc: 0.7598 - categorical_accuracy: 0.7598 - val_loss: 0.7476 - val_acc: 0.7481 - val_categorical_accuracy: 0.7481
Epoch 83/100
1563/1563 [==============================] - 151s 96ms/step - loss: 0.7476 - acc: 0.7559 - categorical_accuracy: 0.7559 - val_loss: 0.7143 - val_acc: 0.7830 - val_categorical_accuracy: 0.7830
Epoch 84/100
1563/1563 [==============================] - 152s 97ms/step - loss: 0.7488 - acc: 0.7583 - categorical_accuracy: 0.7583 - val_loss: 0.7067 - val_acc: 0.7677 - val_categorical_accuracy: 0.7677
Epoch 85/100
1563/1563 [==============================] - 150s 96ms/step - loss: 0.7581 - acc: 0.7534 - categorical_accuracy: 0.7534 - val_loss: 0.6504 - val_acc: 0.7855 - val_categorical_accur

In [45]:
score = model_best.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 7s 681us/step
Test loss: 0.764634818888
Test accuracy: 0.7648


**• You are expected to provide a recommendation for the best model you would recommend for classification. Which model (with parameter values) would you choose and why?**

Because the tests above, I'd like to build a model with 100 epochs, batch size 32, 512 neurons in every layer, 3 layers, 0.0002 learning rate, activation function 'relu' and dropout rate 0.1.

**• Comment on how good your model is ? Does it overfit/underfit data ? What could you do to improve the model?**

My best model fits data well but it can be still improved. Because the testing accuracy is about 0.76 and the loss is 0.76. These two parameters are very close to training results. My model underfits data because the loss of model is still a high value and can be decreased.

I think I can improve my model by doing follows:
1. Add more features. If the under-fitting is due to insufficient features, I can add more feature parameters.
2. Increase model complexity. If the model is too simple to handle complex tasks. More complexity can be used to reduce the regularization coefficients. 
3. I can test more parameters and analyze how each parameter influence the result. For example, I can train model with more epochs, or more layers.